<a href="https://colab.research.google.com/github/abdealijaroli/nlp-spell-checker/blob/main/Spell_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Project - Spell Checker


#### Trinav Rattan (19BCE0493)
#### Pallavi Mishra(19BCB0119)
#### Abdeali Jaroli(19BCE0190)
#### Ayushman Biswari(19BCE0214)
#### Naveen Murali(19BDS0157)




```
`# This is formatted as code`
```



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries


In [ ]:
# Importing all required libraries for this task.
import nltk
import keras
nltk.download('punkt')
from nltk.util import ngrams
from nltk.metrics.distance import edit_distance
from nltk.corpus import words
from nltk.tokenize import RegexpTokenizer
from itertools import chain
import json
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import PunktSentenceTokenizer
from nltk.stem import *
from nltk.corpus import wordnet as wn
import time
from tqdm import tqdm
from difflib import SequenceMatcher
lemmatizer = WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Parsing the data

In [ ]:
def parsing(sent):  
    """Parsing the sentence to corrected and original and storing in the dictionary."""
    loriginal = []
    lcorrected = []
    indexes = []
    cnt = 0
    
    for i in sent:
        if '|' in i:
            # Splitting the sentence on '|'
            str1 = i.split('|')
            # Previous word to '|' is storing in loriginal list.
            loriginal.append(str1[0])
            # Next word to '|' is storing in lcorrected list.
            lcorrected.append(str1[1])
            #Noting down the index of error.
            indexes.append(cnt)
        
        else:
            # If there is no '|' in sentence, sentence is stored in loriginal and lcorrected as it is.
            loriginal.append(i)
            lcorrected.append(i)
        cnt = cnt+1
        
    #Loading to loriginal, lcorrected and index list to dictionary.      
    dictionary = {'original': loriginal, 'corrected': lcorrected, 'indexes': indexes}
    
    return dictionary

# Preprocessing the data

In [ ]:
def preprocessing():
    """Loading the data from 'holbrook.txt' and passing to parsing function to get parssed sentences. 
    Returning the whole dictionary as data."""
    data = []
    
    # Reading the txt file
    text_file = open("holbrook.txt")
    lines = []
    for i in text_file:
        lines.append(i.strip())
    
    # Word tokenizing the sentences
    sentences = [nltk.word_tokenize(sent) for sent in lines]
    
    # Calling a parse function to get corrected, original sentences.
    for sent in sentences:
        data.append(parsing(sent))
    
    return data

#Calling preprocessing function
data = preprocessing()

# Testing
print(data[2])
assert(data[2] == {
 'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'],
 'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'],
 'indexes': [9]
})

# Splitting the data to test 100 lines and remaining training lines
test = data[:100]
train = data[100:]

{'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 'indexes': [9]}


#Splitting into train and test set

In [ ]:
# Splitting the data to test - first 100 lines and remaining training lines
def test_train_split():
    """Splitting the data to test - first 100 lines and remaining training lines."""
    test = data[:100]
    train = data[100:]
    
    # Separating the train original, test original, test corrected and train corrected from dictionary to list.
    train_corrected = [elem['corrected'] for elem in train]
    tokenizer = RegexpTokenizer(r'\w+')
    test_corrected = [elem['corrected'] for elem in test]
    test_original = [elem['original'] for elem in test]
    
    # Removing all special characters from the list.
    test_original = [tokenizer.tokenize(" ".join(elem)) for elem in test_original]
    test_corrected = [tokenizer.tokenize(" ".join(elem)) for elem in test_corrected]
    train_corrected = [tokenizer.tokenize(" ".join(elem)) for elem in train_corrected]
    
    return test_corrected, test_original, train_corrected

# Test and Training data.
test_corrected, test_original, train_corrected = test_train_split()

# Importing Edit Distance function from nltk

In [ ]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


# Getting candidates for replacing a misspelled word

In [ ]:
def get_candidates(token):
    
    """Get nearest word for a given incorrect word."""
    doc = []

    for i in train_corrected:
        doc.append(" ".join(i))

    doc = " ".join(doc)
    doc = nltk.word_tokenize(doc)
    unig_freq = nltk.FreqDist(doc)
    unique_words = list(unig_freq.keys())

    # Calculate distance between two words
    s = []
    for i in unique_words:
        t = edit_distance(i, token)
        s.append(t)
    
    # Store the nearest words in ordered dictionary
    dist = dict(zip(unique_words, s))
    dist_sorted = dict(sorted(dist.items(), key=lambda x:x[1]))
    minimal_dist = list(dist_sorted.values())[0]
    
    keys_min = list(filter(lambda k: dist_sorted[k] == minimal_dist, dist_sorted.keys()))
    
    return keys_min

print(get_candidates("minde"))

['mine', 'mind']


In [ ]:
# This is to calculate unigram and bigram probabilities in correct function
doc = []

for i in train_corrected:
    doc.append(" ".join(i).lower())

doc = " ".join(doc)
doc = nltk.word_tokenize(doc)
unig_freq = nltk.FreqDist(doc)
unique_words = list(unig_freq.keys())

cf_biag = nltk.ConditionalFreqDist(nltk.bigrams(doc))
cf_biag = nltk.ConditionalProbDist(cf_biag, nltk.MLEProbDist)

VBG = Verb Gerund (V-ing) <br>
VBD = Verb Past Tense (V-ed) <br>
VBN = Verb Past Participle (V-ed) <br>

In [ ]:
def tense(suggestion, sentence):    
    """Tense Detection"""
    tag = dict(nltk.pos_tag(sentence)).values()
    past_tense = ['VBN', 'VBD']
    conti_tense = ['VBG']
    
    # If sentence is past tense append ed and check if it is valid word
    if any(x in tag for x in past_tense):
        sug = []
        for a in suggestion:
            if a.lower()+'ed' in unique_words:
                sug.append(a+'ed')
        for aelem in sug:
            suggestion.append(aelem)
            
    # If sentence is continuous tense append ing and check if it is valid word
    if any(x in tag for x in conti_tense):
        sug = []
        for b in suggestion:
            if b.lower()+'ing' in unique_words:
                sug.append(b+'ing')
        for belem in sug:
            suggestion.append(belem)
        
    return suggestion 


def named_entity(sentence):
    """Named Entity Detection using nltk.pos_tag and nltk.ne_chunk"""
    l = []
    for chunk in nltk.ne_chunk(nltk.pos_tag(sentence)):
        # If any named tag like PERSON, ORGANIZATION or GEOLOCATION append to list.
        if hasattr(chunk, 'label'):
            l.append(' '.join(c[0] for c in chunk))

    
    if len(l) != 0:
        l = " ".join(l)
        l = l.split(" ")
        
    return l


def word_forms_new(suggest):
    """Taking different forms of words using derivationally related forms"""
    sug_form = []
    for w in suggest:
        forms = set()
        for i in wn.lemmas(w):
            forms.add(i.name())
            for j in i.derivationally_related_forms():
                forms.add(j.name())
        
        for a in list(forms):
            sug_form.append(a)
    
    for q in sug_form:
        suggest.append(q)
    
    word_forms = []
    [word_forms.append(i) for i in suggest if not i in word_forms]
    return word_forms


def conditions(corrected, cr_ind):
    """Common word - Oclock is not detecting. Hence handling manually but not necessary"""
    
    if 'oclock' in corrected:
        ind = corrected.index('oclock')
        corrected = list(map(lambda x: str.replace(x, "oclock", "clock"), corrected))
        corrected.insert(ind, 'o')
        return corrected
        
    return corrected

def sentence_sentence_similarity(sentence1):
    """Sentence - Sentence similarity using sequence matcher"""
    correc = []
    for d in train_corrected:
        ratio = SequenceMatcher(None, " ".join(d), " ".join(sentence1)).ratio()
        if ratio > 98:
            correc.append(d)
    
    if len(correc) == 1:
        return correc[0]
    else:
        return []


In [ ]:
def is_stemmed_not_in_dict(i):
  stemmer = PorterStemmer()
  return stemmer.stem(i) not in wn.words()

def is_lemmatized_not_in_dict(i):
  return lemmatizer.lemmatize(i.lower()) not in unique_words

def is_word_not_in_known_list(i, n_en, sts):
  return all([i.lower() not in source for source in [n_en, sts, wn.words()]]) 


def check_invalid(i, sts, sentence):
  n_en = named_entity(sentence)
  return all([is_stemmed_not_in_dict(i), is_lemmatized_not_in_dict(i), 
              is_word_not_in_known_list(i, n_en, sts), not i.isdigit()])


def perform_bigram_distr(suggestion, cnt, sentence):
  prob = []
  # Bigram probabilities
  for sug in suggestion:

      # Check the misspelled word is first or last word of the sentence
      if ((cnt != 0) and (cnt != len(sentence) - 1)):

          try:
              p1 = cf_biag[sug.lower()].prob(sentence[cnt + 1].lower())
              p2 = cf_biag[corrected[len(corrected) - 1].lower()].prob(sug.lower())
              p = p1 * p2
              prob.append(p)
          except:
              prob.append(0)

      else:
          # If mispelled word is last word of a sencence take probaility of previous word
          if cnt == len(sentence) - 1:
              try:
                  p2 = cf_biag[corrected[len(corrected) - 1].lower()].prob(sug.lower())
                  prob.append(p2)
              except:
                  prob.append(0)


          elif cnt == 0:
              # If mispelled word is first word of a sencence take probaility of next word
              try:
                  p1 = cf_biag[sug.lower()].prob(sentence[cnt + 1].lower())
                  prob.append(p1)
              except:
                  prob.append(0)

  return prob

# Main function - correct()



1.   Stemming and Lemmatizaion
2.   Sentence Sentence Similarity
3. Named Entity Recgn
4. Min Edit Distance
5. Tense Correction
6. Inclusion of alternative word forms
7. Bi gram Probability 



In [ ]:
def correct(sentence):
    sts = ['oclock']
    corrected = []
    cnt = 0
    indexes = []
    #To check stemmed word in dictonary or not
    stemmer = PorterStemmer()
    status = 0
    #This will extract all named entities of a sentence
    n_en = named_entity(sentence)
    
    for i in sentence:

        # Check for sentence similarity
        corr = sentence_sentence_similarity(i)
        if len(corr) == 1:
            return corr
        # Ignoring digits like page number and lemmatizing the word and check 
        # if it is present in dictionary and use words.words() for word validation.
       
        elif check_invalid(i, sts, sentence):
            indexes.append(cnt)
            if len(get_candidates(i)) > 1:
                # Get words forms, tense detection for suggested sentence
                suggestion = get_candidates(i)
                suggestion = tense(suggestion, sentence)
                wd_fms = word_forms_new(suggestion)
                suggestion = wd_fms

                prob = perform_bigram_distr(suggestion, cnt, sentence)
              
                if len(suggestion[prob.index(max(prob))]) > 1:
                    corrected.append(suggestion[prob.index(max(prob))])
                else:
                    corrected.append(suggestion[prob.index(max(prob))])

            else:
                corrected.append(get_candidates(i)[0])

        else:
            corrected.append(i)

        cnt = cnt+1
        # Manula hadling 'Oclock'
        corrected = conditions(corrected, indexes)
    
    fin = sentence_sentence_similarity(corrected)
    if len(fin) != 0:
        return fin
    else:
        return corrected



correct(['test', 'of', 'goe', 'out', 'some_times'])

['test', 'of', 'go', 'out', 'sometimes']

# Evaluate the Model

In [ ]:
start_time = time.time()
def accuracy(actual_sent, sent_pred):
    """This is based on word to word accuracy calculation. 
    Compares each word with the actual word and calculate accuracy"""
    actual = actual_sent
    predict = correct(sent_pred)
    # If the blank sentence i.e for a blank line predicted
    # is also blank take accuracy as 1

    if len(actual) == 0 and len(predict)==0:
        accuracy = 1.0
    else:
        # Take all predicted words same as actual word 
        #and divide by lenght of sentence

        accuracy = len(set(predict) & set(actual))/len(set(actual))
    
    return accuracy
    
acc = []
for i in tqdm(range(len(test_corrected))):
    acc.append(accuracy(test_corrected[i], test_original[i]))

print("\nAverage Accuracy of words in each sentence: ", 
      round(sum(acc)/len(acc)*100, 4), "%")
elapsed_time = time.time() - start_time
print("Elapsed Time is: ", elapsed_time)

100%|██████████| 100/100 [03:26<00:00,  2.07s/it]


Average Accuracy of words in each sentence:  89.2142 %
Elapsed Time is:  206.9772436618805


# Testing code

In [ ]:
def test(sent):
    print("\nOriginal: " + sent)
    print("Corrected: " + ' '.join(correct(sent.split())))
 
test("Ram is goooood")
test("Adam is whitr in color")
test("look ahedd")
test("NLP is my favourite courde")
test("I am hapoy")





Original: Ram is goooood
Corrected: Ram is good

Original: Adam is whitr in color
Corrected: Adam is white in color

Original: look ahedd
Corrected: look ahead

Original: NLP is my favourite courde
Corrected: NLP is my favourite course

Original: I am hapoy
Corrected: I am happy
